In [3]:
import os
import time
from bs4 import BeautifulSoup
from playwright.sync_api import sync_playwright
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

SEASONS = list(range(2024, 2026))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            def fetch():
                with sync_playwright() as p:
                    browser = p.firefox.launch()
                    page = browser.new_page()
                    page.goto(url)
                    print(page.title())
                    content = page.inner_html(selector)
                    browser.close()
                    return content

            with ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(fetch)
                html = future.result()

        except Exception as e:
            print(f"Error fetching {url}: {e}")
            continue
        else:
            break
    return html

def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = get_html(url, "#content .filter")

    if not html:
        print(f"Failed to retrieve data for season {season}")
        return

    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links if 'href' in l.attrs]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])

        if os.path.exists(save_path):
            print(f"File already exists: {save_path}, skipping...")
            continue

        html = get_html(url, "#all_schedule")

        if html:
            with open(save_path, "w+", encoding="utf-8") as f:
                f.write(html)
            print(f"Saved: {save_path}")
        else:
            print(f"Failed to retrieve: {url}")

for season in SEASONS:
    scrape_season(season)

standings_files = os.listdir(STANDINGS_DIR)

def scrape_game(standings_file):
    with open(standings_file, 'r', encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        scrape_game(filepath)


ModuleNotFoundError: No module named 'playwright'